# 虚拟销售系统 Workshop：实时性与专业性的工程解决方案

## 核心目标

解决虚拟销售/客服系统中的**三个独立工程问题**：

- **≤1500ms 端到端延迟** - 保证实时交互体验
- **30轮内无显著幻觉的专业对话** - 确保回答准确可靠
- **40-60秒长语音的准确理解** - 完整捕捉客户需求

---

## Workshop 结构

```
导入部分 (总)
    ↓
Block 1: 模型选型与延迟分析
    ↓
Block 2: RAG 异构数据融合
    ↓
Block 3: 长输入处理
    ↓
完整 Pipeline 演示
    ↓
总结与讨论 (总)
```

---

## 为什么不讲 MoE？

### 误区澄清

原始需求提到：LLM 专业度提升方案，使用 MoE 路由架构（按行业或产品类型划分）

**但实际上**：
- ❌ **误区1**：不需要自己训练 MoE 模型——Qwen 等模型已内置多专家机制
- ❌ **误区2**：MoE 路由**不能根本解决幻觉问题**——幻觉源于知识不足，而非模型选择
- ✅ **正确理解**：基础模型 + 高质量知识库 + 工程优化 = 核心解决方案

### 本 Workshop 的视角

我们关注的是**工程设计而非模型黑盒**：
1. **模型选择是手段**：选合适规模的 Qwen，确保延迟可控
2. **知识库是关键**：用 RAG 从异构数据精准检索，防止幻觉
3. **处理流程是保障**：长输入分段、完整理解，确保回答准确

**MoE 的真实位置**：如果上述三个环节都优化后，系统仍存在专业度不足的问题，**那时**才考虑 MoE 作为后续优化。但绝大多数场景下，这三个环节就已经足够了。

---

## Part 0: 环境初始化

### 检查 vLLM 服务

本 Workshop 使用本地 vLLM 服务：
- **Qwen3-8B** @ localhost:8000 (轻量级任务)
- **Qwen3-14B** @ localhost:8001 (主对话生成)

In [ ]:
# 检查 vLLM 服务是否运行
import requests

def check_vllm_service(port, model_name):
    """检查 vLLM 服务状态"""
    try:
        response = requests.get(f"http://localhost:{port}/v1/models", timeout=3)
        if response.status_code == 200:
            print(f"✓ {model_name} 服务运行正常 (端口 {port})")
            return True
        else:
            print(f"✗ {model_name} 服务异常 (端口 {port}): {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"✗ {model_name} 服务未运行 (端口 {port})")
        return False

print("=" * 60)
print("检查 vLLM 服务状态")
print("=" * 60)

service_8b = check_vllm_service(8000, "Qwen3-8B")
service_14b = check_vllm_service(8001, "Qwen3-14B")

if not service_8b or not service_14b:
    print("\n⚠️  vLLM 服务未完全启动")
    print("   → 请运行下面的 bash cell 启动服务")
else:
    print("\n✓ 所有服务运行正常，可以跳过下面的启动步骤")

### (可选) 检查GPU状态

如果需要启动vLLM，先检查GPU是否可用：

In [ ]:
%%bash
# 检查GPU状态
nvidia-smi --query-gpu=index,name,memory.total,memory.used,memory.free --format=csv

### 启动 vLLM 服务（如果未运行）

**注意**：这个cell会在后台启动vLLM服务，大约需要1-2分钟加载模型。

启动后请等待约1-2分钟，然后重新运行上面的检查cell确认服务已启动。

In [ ]:
%%bash --bg
# 后台启动vLLM双模型服务
bash scripts/start_dual_vllm_services.sh

### 导入核心库

导入项目所需的核心库（OpenAI客户端、RAG组件、知识库数据）。

In [ ]:
# 导入核心库
import sys
from pathlib import Path

# 添加项目根目录到路径
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root))

from openai import OpenAI
from rag_utils import (
    EmbeddingService, RerankingService, VectorIndex,
    BM25, hybrid_search, build_rag_context
)
from data.fictional_knowledge_base import FICTIONAL_DOCUMENTS
from data.company_graph import convert_all_companies_to_documents

print("✓ 核心库导入成功")

### 初始化 RAG 服务

这一步会初始化所有必要的服务组件（vLLM客户端、Embedding、Reranking、向量索引、BM25索引）。

**注意**：这一步耗时较长（约30-60秒），因为需要：
- 连接vLLM服务
- 加载54个知识库文档
- 构建向量索引（调用Embedding API）
- 构建BM25索引

In [ ]:
# 初始化服务
print("初始化 RAG 服务...")

# 创建 vLLM 客户端
client_8b = OpenAI(api_key="EMPTY", base_url="http://localhost:8000/v1")
client_14b = OpenAI(api_key="EMPTY", base_url="http://localhost:8001/v1")
print("✓ vLLM 客户端初始化完成")

# 初始化 RAG 组件
embedding_svc = EmbeddingService()
reranking_svc = RerankingService()
vector_idx = VectorIndex(embedding_svc)
print("✓ RAG 组件初始化完成")

# 加载知识库
all_docs = FICTIONAL_DOCUMENTS + convert_all_companies_to_documents()
print(f"✓ 加载知识库: {len(FICTIONAL_DOCUMENTS)} 个业务文档 + {len(convert_all_companies_to_documents())} 个公司文档")

# 构建向量索引
vector_idx.add_documents(all_docs)
print(f"✓ 向量索引构建完成: 共 {len(all_docs)} 个文档")

# 构建 BM25 索引
corpus = [f"{doc['title']} {doc['content']}" for doc in all_docs]
bm25_idx = BM25(corpus)
print("✓ BM25 索引构建完成")

print("\n=" * 60)
print("所有服务初始化完成，准备开始实验")
print("=" * 60)

---

## Part 1: Block 1 - 模型选型与延迟分析

### 问题
应该选多大的 Qwen 模型才能满足 ≤500ms 推理时间？

### 因果链
500ms 推理约束 → 模型规模 → 硬件需求

### 解决思路
1. **Qwen 系列对标**: 3B/7B/14B/72B 参数规模选择
2. **推理速度计算**: 单 token 生成时间 × 平均回答长度 = 推理延迟
3. **实验验证**: 在目标硬件上跑 benchmark

### 理论延迟估算

通过简单计算评估不同参数规模模型的推理延迟（理论值）。

In [ ]:
# 理论延迟估算
import numpy as np

models = {
    "Qwen3-3B": {"params": 3e9, "tokens_per_sec": 100},
    "Qwen3-7B": {"params": 7e9, "tokens_per_sec": 50},
    "Qwen3-14B": {"params": 14e9, "tokens_per_sec": 25},
    "Qwen3-72B": {"params": 72e9, "tokens_per_sec": 8},
}

avg_response_tokens = 50  # 平均回答长度

print("推理延迟估算 (假设平均回答50个token):")
print("-" * 50)
for name, spec in models.items():
    inference_time = avg_response_tokens / spec["tokens_per_sec"]
    status = "✓ 可行" if inference_time <= 1.5 else "✗ 超限"
    print(f"{name}: {inference_time:.3f}s {status}")

print("\n注意: 这是理论估算，实际测试结果见下文实验1")

### 实验1结果：模型对比测试

**实验时间**: 2024-12-13  
**测试代码**: `experiments/test_01_model_comparison.py`  
**数据文件**: `outputs/experiment1_results_llm_scored_200059.json`  

#### 测试模型与性能

| 模型 | 准确性 | 完整性 | 平均延迟(s) | 速度(tok/s) | 评价 |
|------|--------|--------|-------------|-------------|------|
| qwen3-8b | 6.7/10 | 7.7/10 | 21.77 | 4.4 | ❌ 速度最慢，不适合实时 |
| **qwen3-14b** | 6.7/10 | **7.9/10** | **20.63** | **7.4** | ✅ **最佳平衡点** |
| qwen3-32b | 6.7/10 | 7.8/10 | 35.22 | 9.0 | ⚠️ 速度快但初始延迟高 |

#### 关键发现

1. **qwen3-14b 是实时交互的最佳选择**：完整性最高（7.9/10）、延迟最低（20.63秒）
2. **qwen3-8b 不适合实时场景**：速度仅4.4 tok/s，但可作为辅助模型用于快速判断任务
3. **双模型架构**：8B（RAG判断、输入完整性检测）+ 14B（主对话生成）

In [ ]:
# 加载实验1结果
import json
import pandas as pd

exp1_file = project_root / 'outputs' / 'experiment1_results_llm_scored_200059.json'
with open(exp1_file, 'r', encoding='utf-8') as f:
    exp1_data = json.load(f)

# 统计各模型性能
models_stats = {}
for result in exp1_data['results']:
    model = result['model']
    if model not in models_stats:
        models_stats[model] = {
            'accuracy': [],
            'completeness': [],
            'latency': [],
            'speed': []
        }
    
    scores = result['llm_scores']
    models_stats[model]['accuracy'].append(scores['accuracy'])
    models_stats[model]['completeness'].append(scores['completeness'])
    models_stats[model]['latency'].append(result['latency'])
    models_stats[model]['speed'].append(result['speed'])

# 计算平均值
print("实验1 - 模型性能对比")
print("=" * 80)
for model, stats in models_stats.items():
    print(f"\n{model}:")
    print(f"  准确性: {np.mean(stats['accuracy']):.1f}/10")
    print(f"  完整性: {np.mean(stats['completeness']):.1f}/10")
    print(f"  平均延迟: {np.mean(stats['latency']):.2f}s")
    print(f"  平均速度: {np.mean(stats['speed']):.1f} tok/s")

---

## Part 2: Block 2 - RAG 异构数据融合

### 问题
结构化企业图谱数据和非结构化业务文档怎么整合，才能有效降低幻觉？

### 因果链
异构数据（结构化+非结构化）→ 分别处理 → 统一检索 → 降低幻觉

### 解决思路
1. **数据源特征分析**: 结构化（图谱）vs 非结构化（文档）
2. **分开存储的优势**: 精确检索 + 语义检索
3. **融合检索策略**: BM25 + Dense Vector + RRF + Reranking
4. **幻觉抑制机制**: 来源标记 + 覆盖度评估

In [ ]:
# RAG 异构数据融合演示（简化版）

# 1. 结构化数据示例（企业图谱）
structured_data = {
    "company": "西门子",
    "industry": "工业自动化",
    "solutions": ["PLC控制", "工业互联网", "数字化转型"],
    "clients": ["宝马", "大众", "西门子中国"],
}

# 2. 非结构化数据示例（业务文档）
unstructured_docs = [
    "西门子 Siemens S7-1200 PLC 是面向中小型应用的高性能控制器...",
    "工业4.0 解决方案可以帮助传统制造业实现数字化转型...",
    "云平台支持实时监控和远程诊断功能...",
]

# 3. 用户问题
user_query = "西门子的 PLC 产品有什么特点？"

# 4. 分别检索
print("=== 检索流程演示 ===\n")

# 结构化检索（精确）
print("【结构化检索】")
if "PLC" in user_query:
    for solution in structured_data["solutions"]:
        if "PLC" in solution:
            print(f"✓ 匹配: {solution}")

# 向量检索（模拟）
print("\n【向量检索】")
query_keywords = ["PLC", "特点", "功能"]
for i, doc in enumerate(unstructured_docs):
    match_score = sum(1 for kw in query_keywords if kw in doc) / len(query_keywords)
    if match_score > 0:
        print(f"文档 {i+1}: {doc[:50]}... (相似度: {match_score:.2f})")

# 5. 融合结果
print("\n【融合结果】")
rag_context = f"""
结构化信息：西门子的核心解决方案包括 {', '.join(structured_data['solutions'])}
文档补充：{unstructured_docs[0][:50]}...
"""
print(rag_context)
print("\n→ LLM 基于上述背景知识回答，避免纯粹幻觉")

### 实验2结果：混合RAG策略验证

**实验时间**: 2024-12-15 / 2024-12-20  
**测试代码**: `experiments/test_02_*.py`  

#### 4种检索策略对比

| 方案 | 策略 | 检索时间 | 评价 |
|------|------|----------|------|
| A | Business-only Dense | 934ms | ❌ 仅业务文档，缺少公司信息 |
| B | Company-only BM25 | 0.23ms | ⚠️ 仅公司文档，缺乏产品细节 |
| C | All Dense | 1079ms | ❌ 语义检索，精确匹配不足 |
| **D** | **Hybrid (BM25+Dense+RRF+Rerank)** | 935ms | ✅ **综合效果最佳** |

#### 最终性能指标（方案D + 本地vLLM）

| 指标 | 结果 |
|------|------|
| **通过率** | **100%** (8/8) ✅ |
| **平均召回率** | **89.2%** |
| 平均检索时间 | 1.44秒 |
| 平均生成时间 | 0.70秒 |
| **平均总延迟** | **2.14秒** |

#### 关键发现

1. **混合检索策略效果最优**: BM25关键词匹配 + Dense语义理解 + Reranking精排
2. **中文分词优化显著**: 自定义jieba词典，召回率从70%提升至89.2%
3. **本地vLLM降低延迟**: 总延迟仅2.14秒，适合实时交互

In [ ]:
# 真实混合检索演示
test_query = "星辰金融集团想做实时风控，应该推荐哪个产品？"

print(f"查询: {test_query}")
print("=" * 60)

# 执行混合检索
rag_results, debug_info = hybrid_search(
    test_query, 
    bm25_idx, 
    vector_idx,
    embedding_svc, 
    reranking_svc, 
    final_top_k=5
)

print(f"\n检索到 {len(rag_results)} 个相关文档:\n")
for i, result in enumerate(rag_results, 1):
    print(f"{i}. [{result['score']:.3f}] {result['title']}")
    print(f"   {result['content'][:100]}...\n")

# 构建 RAG 上下文
context = build_rag_context(rag_results)
print("\n构建的RAG上下文:")
print("-" * 60)
print(context[:300] + "...")

---

## Part 3: Block 3 - 长输入处理

### 问题
客户讲了一大堆需求（40-60秒），怎么处理才能既不丢失信息，又避免模型混乱和幻觉？

### 因果链
40-60秒长语音 → 语义分段 → 独立理解 + 上下文保留 → 完整回答

### 解决思路
1. **长输入的两大挑战**: Token 长度超限 + 多需求混合
2. **分段策略**: 语义边界检测（非固定长度截断）
3. **处理方式**: 流式 vs 聚合 vs 混合
4. **质量保障**: 关键点提取 + 段间逻辑 + 完整性检查

In [ ]:
# 长输入分段处理演示
import re

# 模拟客户 60 秒的语音转录
long_input = """
我们公司是一家制造企业，主要生产汽车零部件。
目前面临的问题是生产效率低下，产品不良率在 15% 左右。
我们听说西门子的工业控制系统能帮助优化生产流程。
另外，我们的库存管理也很混乱，经常出现过库或缺库的情况。
想问一下，西门子是否有完整的 ERP 加自动化的整体解决方案？
"""

print("=== 长输入分段处理 ===\n")

# 1. 语义分段
sentences = re.split(r'[。？！]', long_input.strip())
sentences = [s.strip() for s in sentences if s.strip()]

print(f"原始输入长度: {len(long_input)} 字符")
print(f"分段数量: {len(sentences)} 个语义单元\n")

# 2. 逐段处理
segments_with_topics = []
for i, sentence in enumerate(sentences):
    # 提取关键信息
    if "效率" in sentence or "不良率" in sentence:
        topic = "生产优化"
    elif "库存" in sentence:
        topic = "库存管理"
    elif "解决方案" in sentence:
        topic = "产品咨询"
    else:
        topic = "背景信息"
    
    segments_with_topics.append({
        "id": i+1,
        "text": sentence,
        "topic": topic
    })

print("【分段结果】")
for seg in segments_with_topics:
    print(f"段 {seg['id']} [{seg['topic']}]: {seg['text']}")

# 3. 关键点提取
print("\n【关键点提取】")
key_points = {
    "企业类型": "汽车零部件制造",
    "主要问题": ["生产效率低", "产品不良率 15%", "库存管理混乱"],
    "咨询方向": "工业控制系统 + ERP 整体方案",
}
for key, value in key_points.items():
    print(f"  {key}: {value}")

# 4. 完整性检查
print("\n【完整性检查】")
all_topics = set(seg["topic"] for seg in segments_with_topics)
print(f"涵盖的话题: {all_topics}")
print("✓ 背景信息完整")
print("✓ 问题点清晰")
print("✓ 咨询需求明确")

print("\n→ 现在 LLM 可以基于这个完整的背景，生成一致的专业回答")

### 实验3结果：渐进式总结与增量RAG

**实验时间**: 2024-12-24  
**测试代码**: `experiments/test_03_v3_server.py`  

#### 4种处理方法对比

| 方法 | 策略 | 总分 | 感知延迟 | 评价 |
|------|------|------|----------|------|
| M1 | Baseline（完整文本） | 87.7/100 | 58.47s | ⚠️ 无压缩，上下文易溢出 |
| M2 | Batch Summary（批量总结） | 72.8/100 | 81.42s | ❌ 延迟最高 |
| M3 | Incremental v2（仅保留最后段落） | 70.2/100 | 61.14s | ❌ 信息丢失 |
| **M4** | **Incremental RAG v3** | **90.2/100** | **60.38s** | ✅ **综合最优** |

#### Method 4 核心优势

1. **最高综合评分**: 90.2/100（信息保留94.6、RAG相关性83.6）
2. **低感知延迟**: 总结和RAG都在用户输入过程中完成
3. **最优压缩效果**: Query压缩至31.3%，避免上下文溢出
4. **智能RAG**: 增量检索 + 相关度过滤（cosine > 0.6）+ 文档去重

#### 处理流程示意

```
用户说话过程 ────────────────────────────┐
                                       ↓
M1: 无处理 ─────────────────→ 等待RAG+生成 (58.47s)

M2: 等待说完 → 批量总结 → RAG+生成 (81.42s)

M3: 边说边总结(后台) ─────→ RAG+生成 (61.14s)
    总结时间: 42.06s (隐藏)

M4: 边说边总结(后台) + 增量RAG ─→ 最终生成 (60.38s) ⭐
    总结时间: 45.35s (隐藏)
    RAG时间: 3.66s (分散)
```

In [ ]:
# 加载实验3结果
exp3_file = project_root / 'outputs' / 'experiment3_v3_server_results_20251224_131035.json'
with open(exp3_file, 'r', encoding='utf-8') as f:
    exp3_data = json.load(f)

# 展示各方法的综合评分
print("实验3 - 长音频处理方法对比")
print("=" * 80)

for method_name, method_data in exp3_data['summary'].items():
    print(f"\n{method_name}:")
    avg_scores = method_data['average_scores']
    print(f"  综合评分: {avg_scores['overall']:.1f}/100")
    print(f"  信息保留率: {avg_scores['information_preservation']:.1f}")
    print(f"  噪音过滤率: {avg_scores['noise_filtering']:.1f}")
    print(f"  RAG相关性: {avg_scores['rag_relevance']:.1f}")
    print(f"  回复质量: {avg_scores['response_quality']:.1f}")
    print(f"  感知延迟: {method_data['avg_total_latency']:.2f}s")

---

## Part 4: 完整 Pipeline 演示

### 三个解决方案的组合效应

```
参数选型 (Block 1)
     ↓
能否在 500ms 内推理？
     ↓
+ RAG 异构融合 (Block 2)
     ↓
是否能获取准确知识？
     ↓
+ 长输入分段 (Block 3)
     ↓
能否理解完整需求？
     ↓
→ 实现实时+专业的系统
```

In [ ]:
# 端到端 Pipeline 真实调用
import time

# 真实场景测试
customer_input = """
我们是一家金融科技公司，想要构建实时风控系统。
目前使用传统规则引擎，但响应速度慢、误报率高。
听说 TechFlow 有相关产品，能帮我们分析一下吗？
"""

print("=" * 60)
print("🎤 客户输入:")
print(customer_input)
print("=" * 60)

# Step 1: 长输入分段（这里输入较短，无需分段）
print("\n[Step 1] 长输入处理...")
segments = [s.strip() for s in customer_input.strip().split('。') if s.strip()]
print(f"✓ 分成 {len(segments)} 个语义单元")

# Step 2: 混合 RAG 检索
print("\n[Step 2] RAG 异构数据检索...")
rag_start = time.time()
rag_results, debug_info = hybrid_search(
    customer_input, 
    bm25_idx, 
    vector_idx,
    embedding_svc, 
    reranking_svc, 
    final_top_k=3
)
rag_time = time.time() - rag_start
context = build_rag_context(rag_results)
print(f"✓ 从知识库检索到 {len(rag_results)} 条相关信息 (耗时: {rag_time:.3f}s)")

# Step 3: LLM 推理（使用 14B 模型）
print("\n[Step 3] LLM 推理（Qwen3-14B）...")
llm_start = time.time()
response = client_14b.chat.completions.create(
    model="Qwen/Qwen3-14B",
    messages=[
        {"role": "system", "content": "你是TechFlow的智能客服，专业、简洁、实用。"},
        {"role": "user", "content": f"背景知识：{context}\n\n问题：{customer_input}"}
    ],
    temperature=0.7,
    max_tokens=500
)
llm_time = time.time() - llm_start
answer = response.choices[0].message.content

print(f"✓ 推理完成 (耗时: {llm_time:.3f}s)")
print("\n💬 AI 回答:")
print("-" * 60)
print(answer)
print("-" * 60)

# Step 4: 性能统计
print("\n⏱️  性能统计:")
e2e_latency = {
    "ASR": 0.3,  # 估算
    "长输入处理": 0.0,  # 本例无需
    "RAG检索": rag_time,
    "LLM推理": llm_time,
    "TTS": 0.4,  # 估算
}
total = sum(e2e_latency.values())

for step, latency in e2e_latency.items():
    print(f"  {step}: {latency:.3f}s")
print(f"\n端到端总延迟: {total:.3f}s")
print("✓ 符合 ≤1500ms 目标" if total < 1.5 else "⚠️ 超过目标")

---

## Part 5: 总结与讨论

### 关键收获总结

| Block | 核心问题 | 关键输出 |
|-------|---------|----------|
| Block 1 | 用什么参数的 Qwen？| 双模型架构（8B+14B） |
| Block 2 | 如何处理异构数据？| 混合RAG（BM25+Dense+RRF） |
| Block 3 | 长输入怎么不出错？| 渐进式总结+增量RAG |

**最终价值**: 参会者拿走可直接用于生产的技术方案和代码框架

---

### 实施优先级

1. **优先级1**: 选定模型规模，完成硬件评估
2. **优先级2**: 搭建RAG基础设施（混合检索+Reranking）
3. **优先级3**: 集成长输入处理逻辑（渐进式总结）
4. **可选**: 根据实际效果，评估是否需要 MoE 等高级优化

---

### 风险检查清单

- [ ] 推理延迟是否稳定 < 500ms？
- [ ] RAG 检索的精度与召回率是否可接受？
- [ ] 长输入分段是否保留了完整信息？
- [ ] 幻觉频率是否在可控范围内？

---

### 互动环节

- 参会者提出的具体场景讨论
- 针对性的模型选择建议
- Q&A 时间

---

## 参考资料

- **实验结果详情**: [docs/EXPERIMENT3_RESULTS.md](../docs/EXPERIMENT3_RESULTS.md)
- **vLLM部署指南**: [docs/LOCAL_VLLM_GUIDE.md](../docs/LOCAL_VLLM_GUIDE.md)
- **完整系统文档**: [README.md](../README.md)

---

## 附录：vLLM 服务管理

### 查看 vLLM 服务日志

如果服务启动失败或运行异常，可以查看日志：

In [ ]:
%%bash
# 查看 Qwen3-8B 最近的日志
echo "=== Qwen3-8B (端口 8000) 日志 ==="
tail -30 logs/vllm_8b.log

echo ""
echo "=== Qwen3-14B (端口 8001) 日志 ==="
tail -30 logs/vllm_14b.log

### 停止 vLLM 服务

Workshop 结束后，可以停止 vLLM 服务释放 GPU 资源：

In [ ]:
%%bash
# 停止所有vLLM进程
pkill -f "vllm.entrypoints.openai.api_server"
echo "✓ vLLM 服务已停止"